IMPORTS AND CONFIGURATION

In [1]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy

In [2]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\nearIV_dipsersion', 'c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Program Files\\Python312\\Lib\\site-packages']
The backtesting folder was not in the systems path
path after ['c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\nearIV_dipsersion', 'c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Pyt

In [26]:
# from scipy.optimize import minimize
from Modules import TradeAndLogics as TL
from Modules import Plot
from Modules import Data as data
from Modules import Data_Processing as dp
from Modules.enums import Option, LongShort, DB, FNO, Leg, OHLC, PTSL
from Modules import Utility as util

In [4]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\nearIV_dipsersion'

In [5]:
original_stdout = util.const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [6]:
index_symbol = 'BANKNIFTY'
start_date = '2023-07-01'
end_date = '2024-01-01'
expiry_type = 'I'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 1 # mins
# Trade/ Strategy Parameters
buying_delta_threshold_per_lot = 5
selling_delta_threshold_per_lot = 1
notional_vega_buying = 10000 # rs
notional_vega_selling = 10000 # rs
zscore_threshold_long = 3
zscore_threshold_short = -2.5
ICView = LongShort.Long
profit_target = 15000
stop_loss = 10000
# epsilon = 0.1

In [7]:
strategy_desc = f"IC_nearIV"

Strategy Specific Data Dealing and Modifications

In [8]:
def tell_delta_threshold_per_lot(ticker):
    if ticker.intention == LongShort.Long:
        return buying_delta_threshold_per_lot
    return selling_delta_threshold_per_lot

def tell_notional_vega(ticker):
    if ticker.intention == LongShort.Long:
        return notional_vega_buying
    return notional_vega_selling

In [9]:
import Dispersion.DispersionAdjustedFunctionality as daf

In [10]:
basket = daf.RawWeightedPortfolio()
basket.insert('HDFCBANK', 550, 27.04)
basket.insert('ICICIBANK', 700, 23.03)
basket.insert('KOTAKBANK', 400, 11.72)
basket.insert('SBIN', 750, 11.27)
basket.insert('AXISBANK', 625, 11.18)

In [11]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

BACK TO NORMAL

In [15]:
def entry_signal(timestamp):
    if timestamp == entry:
        return True
    return False

def exit_signal(timestamp):
    if TL.isTodayAnyExpiry(timestamp, *portfolio):
        return True
    if timestamp == exit:
        return True
    return False

In [12]:
def get_vega_legs(ticker, timestamp, *Legs):
    vega = 0
    for leg in Legs:
        if leg.Instrument == FNO.FUTURES:
            continue
        greeks = ticker.Greeks(timestamp, leg.Instrument, leg.Strike)
        if greeks is None:
            return None
        vega += greeks['vega'] * leg.Position.value
    return abs(vega)

In [14]:
def get_lots_for_entry(ticker, timestamp, **kwargs):
    logging_information = {}
    legs = Helpers.get_legs_ironfly_WithFarOptionsOfPrice_ATMpriceXfactor(ticker, timestamp, 1, ticker.intention, price_factor)
    vega_ticker_ironfly = get_vega_legs(ticker, timestamp, *legs)
    if vega_ticker_ironfly is None:
        return None, None
    unweighted_vega = vega_ticker_ironfly * ticker.lot_size
    notional_vega = tell_notional_vega(ticker)
    idea_for_weight = ticker.weight
    logging_information = {
        'Lot Size': ticker.lot_size,
        'Weight': ticker.weight,
        'Vega per IronFly': vega_ticker_ironfly,
        'Vega per Lot IronFly': vega_ticker_ironfly * ticker.lot_size,
        'Target Notional Vega': notional_vega * ticker.weight
    }
    
    # if 'vega_neutral' in kwargs and kwargs['vega_neutral']:
    
    if 'ic_neutral' in kwargs and kwargs['ic_neutral'] == True:
        if ticker.is_component:
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    elif 'theta_neutral' in kwargs and kwargs['theta_neutral'] == True:
        strike_ticker, _ = ticker.find_moneyness_strike(timestamp, 0, Option.Call)
        logging_information['ATM Strike'] = strike_ticker
        if ticker.is_component:
            import math
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    lots_ticker = int(np.round(notional_vega * idea_for_weight/ unweighted_vega))
    logging_information['Lots to take Position'] = lots_ticker
    logging_information['Vega Satisfied'] = lots_ticker * unweighted_vega

    print(f">> {ticker.symbol}")
    for info_key, info_value in logging_information.items():
        print(f"  {info_key}: {info_value}")
    return lots_ticker

In [16]:
def take_dispersion_position(timestamp, remarks, ticker, lots):
    legs = []
    legs = Helpers.get_legs_ironfly_WithFarOptionsOfPrice_ATMpriceXfactor(ticker, timestamp, lots, ticker.intention, price_factor)
    for leg in legs:
        key = f'{ticker.symbol}_{leg.id()}'
        if key not in ticker.tokens.keys():
            ticker.tokens[key] = TL.Token(ticker, leg.Instrument, leg.Strike, leg.LegName)
        token = ticker.tokens[key]
        token.add_position(timestamp, lots, leg.Position)
    ticker.take_position(timestamp, remarks, *legs)

In [17]:
def reset_hedging_positions(ticker):
    ticker.hedging = pd.DataFrame(columns = ['date_timestamp', 'lots', 'position', 'futures_price', 'delta_added'])
    ticker.hedging.set_index('date_timestamp', inplace=True)

def reset_trade_positions(ticker):
    ticker.Trades = dp.Trades()
    ticker.tokens = {}

In [18]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        # return True
        if self.active_ptsl:
            self.update_validity(timestamp)
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        index = self.tickers[0]
        if self.triggered_at is None:
            return
        for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
            if abs(z) <= epsilon:
                self.triggered_at = None
                self.active_ptsl = False  
                print(f"PTSL reverted back to mean (<= {epsilon}): Future Signals can now be used for trading")

In [19]:
def squareoff(timestamp, remarks, logging_token_update = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    for ticker in portfolio:
        TL.squareoff_ticker(timestamp, remarks, ticker, logging_token_update)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")

In [20]:
def UpdateDispersionTickers(timestamp, remarks, hedge, logging_token_update, logging_hedging):
    hedge_using_synthetic_futures = True
    for ticker in portfolio:
        delta_threshold_per_lot = tell_delta_threshold_per_lot(ticker)
        TL.HandleUpdate(ticker, timestamp, remarks, hedge, delta_threshold_per_lot, hedge_using_synthetic_futures, logging_token_update, logging_hedging)

In [21]:
def zoom_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    performance = Helpers.zoom_tokens_performance_bar_by_bar(*portfolio, start=start, end=end)
    Plot.save_df_to_excel(performance, file_name)

In [22]:
def visualise_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    result_df = Helpers.get_summary_portfolio(*portfolio, start=start, end=end)

    result_df['IC'] = ic.loc[start:end, 'ic']

    fig = Plot.plot_df(result_df, *(result_df.columns))
    Plot.save_plot(fig, file_name)

In [23]:
def save_dispersion_trades(start, end, file_name):
    trades_dict = {}
    trades_dict['All Trades'] = Helpers.get_trades_portfolio(*portfolio, start=start, end=end)
    for ticker in portfolio:
        trades_dict[ticker.symbol] = Helpers.get_trades_ticker(ticker, start=start, end=end)
    Plot.save_df_to_excel(trades_dict, file_name)

In [24]:
def generate_signal_data(index, index_near, index_next, timestamp):
    index_near.get_iv_at(timestamp)
    index_next.get_iv_at(timestamp)
    index.df_futures.loc[timestamp, 'iv'] = Helpers.get_normallized_iv(timestamp, index_near, index_next)
    index.get_ic_at(timestamp)
    window = 500
    idx = index.df_futures.index.get_loc(timestamp)
    window_data = index.df_futures['ic'].iloc[idx - window + 1 : idx + 1]
    rolling_mean = window_data.mean()
    rolling_std = window_data.std()
    zscore = (index.df_futures.loc[timestamp, 'ic'] - rolling_mean) / rolling_std
    index.df_futures.loc[timestamp, 'zscore'] = zscore
    return

In [ ]:
# positions = {}
# reset_hedging_positions(index)
# reset_trade_positions(index)
# for component in index.components.values():
#     reset_hedging_positions(component)
#     reset_trade_positions(component)


# PTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())
# current_position = None

# file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w')
# sys.stdout = file
# trade_start_date = None
# trade_end_date = None
# trade_count = 0

# for time_ix, timestamp in enumerate(index.df_futures.index):
#     print()
#     print(f"Timestamp: {timestamp}")
#     print(f"IC = {index.df_futures.loc[timestamp, 'ic']} | Z-Score = {index.df_futures.loc[timestamp, 'zscore']} | Filled IC = {index.df_futures.loc[timestamp, 'filled_ic']}")
#     #######################################################################################################################################################
#     PTSL.update_validity(timestamp)
#     if TL.check_existing_position(index):
        
#         print(f"Existing Position Check: TRUE | Dispersion position: {current_position}")
        
#         # EXPIRY DAY MANUAL SQUAREOFF
#         if isTodayAnyExpiry(timestamp, index):
#             print("************  EXPIRY DAY, MANUAL SQUARE OFF ************  ")
#             UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, False, False)
#             squareoff(time_ix, timestamp, 'Expiry Day, Manual Square Off', False)
#             trade_end_date = timestamp
#             trade_count+=1
#             zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
#             visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
#             continue
            
            
#         # PROFIT TARGET AND STOP LOSS SQUAREOFF
#         ptsl_result = PTSL.status(timestamp) 
#         if ptsl_result != "good_to_go":
#             print("************  PTSL Trigger hit *********** ")
#             print(f"{PTSL.nature} Square off at NetPnl of {PTSL.pnl_last_trade}")
#             UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
#             squareoff(time_ix, timestamp, f'{ptsl_result} Square Off', True)
#             trade_end_date = timestamp
#             trade_count+=1
#             zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
#             visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
#             continue
            
            
#         # REVERSE SIGNAL SQUAREOFF
#         squareoff_signal = (current_position == LongShort.Long and exit_signal(timestamp)) or (current_position == LongShort.Short and entry_signal(timestamp))
            
#         if squareoff_signal:
#             print("Entered squareoff signal block")
#             print("************  SquareOff Reverse zscore Signal hit. ************ ")
#             UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
#             squareoff(time_ix, timestamp, 'ZScore Reversal Square Off (Entry in next timestamp/ signal)')
#             trade_end_date = timestamp
#             trade_count+=1
#             zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
#             visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
#             continue
            
#         # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
#         UpdateDispersionTickers(time_ix, timestamp, 'Delta Hedging', True, True, True)
#         print("========================================================================================================================================================================")
#         continue
#     #######################################################################################################################################################
    
    
#     #######################################################################################################################################################
#     if entry_signal(timestamp):
#         print("************  LONG DISPERSION SIGNAL ************  ")
#         UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
#         if isTodayAnyExpiry(timestamp, index):
#             print("Expiry Day today. Trading on Expiry Day is not Allowed")
#             continue

#         if not PTSL.is_valid(timestamp):
#             print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
#             # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
#             continue
        
#         try:
#             lots_index, lots_components = get_lots_for_dispersion(LongShort.Short, index, timestamp, True, False, False, False, notional_vega)
#             # print(f"Index Lots: {lots_index} || Components Lots: {lots_index, lots_components}")

#             if lots_index == 0:
#                 print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
#                 # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
#                 continue

#             take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', index, LongShort.Short, lots_index)
#             for (component, lots_component) in zip(index.components.values(), lots_components):
#                 take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', component, LongShort.Long, lots_component)
#             PTSL.fresh_trade(timestamp)
#             current_position = LongShort.Long
#             trade_start_date = timestamp
#             print(f"Long dispersion (Short Index Long Components) trade executed")
#         except Exception as e:
#             print(f"Couldn't Enter a Long Dispersion Position: {e}")  

#         continue
#     #######################################################################################################################################################


#     #######################################################################################################################################################
#     elif exit_signal(timestamp):
#         print("************  SHORT DISPERSION SIGNAL ************  ")
#         UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
#         if isTodayAnyExpiry(timestamp, index):
#             print("Expiry Day today. Trading on Expiry Day is not Allowed")
#             continue

#         if not PTSL.is_valid(timestamp):
#             print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
#             # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
#             continue

#         try:
#             lots_index, lots_components = get_lots_for_dispersion(LongShort.Long, index, timestamp, True, False, False, False, notional_vega)
#             # print(f"lots_index, lots_components: {lots_index, lots_components}")

#             if lots_index == 0:
#                 print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
#                 # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
#                 continue
            
#             take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', index, LongShort.Long, lots_index)
#             for (component, lots_component) in zip(index.components.values(), lots_components):
#                 take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', component, LongShort.Short, lots_component)
#             PTSL.fresh_trade(timestamp)
#             current_position = LongShort.Short
#             trade_start_date = timestamp
#             print(f"Short dispersion (Long Index Short Components) trade")
#         except Exception as e:
#             print(f"Couldn't Enter a Short Dispersion Position: {e}")  
#         continue
#     #######################################################################################################################################################

#     UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
#     print("========================================================================================================================================================================")
    

# file.close()
# sys.stdout = original_stdout.value

In [25]:
def get_index(start, end):
    constituents = {}
    ohlc = OHLC.close
    index = daf.ticker(index_symbol, index_lot_size, True, start, end, expiry_type, True, timeframe, True, 0.1)
    index.initializeDispersion(constituents, False, 1)
    index.set_ohlc(ohlc)
    index.set_intention(ICView)
    for stock in basket.Symbols():
        constituents[stock] = daf.ticker(stock, basket.LotSize(stock), True, start, end, expiry_type, True, timeframe, False, 0.1)
        constituents[stock].initializeDispersion({}, True, basket.Weight(stock))
        constituents[stock].set_ohlc(ohlc)
        constituents[stock].set_intention(ICView.opposite())
    portfolio = [index, *[component for component in index.components.values()]]
    return index, portfolio

In [ ]:
def trading_loop(index, portfolio):
    file = open(fr'{folder_path}\LOGS_Trading_Logic.txt', 'w', buffering = 1)
    sys.stdout = file

    for ticker in portfolio:
        ticker.reset_trades()

    TrackPTSL = DispersionPTSL(profit_target, stop_loss, *portfolio)
    timestamps = index.timestamps
    current_position = None
    trade_start_date, trade_end_date, trade_count = None, None, 0
    for timestamp in timestamps:
        print()
        print(f"Timestamp: {timestamp}")
        #######################################################################################################################################################
        if TL.check_existing_position(index):
            
            print(f"Existing Position Check: TRUE | IC position: {current_position}")
            
            # 3:20 (Market Close) SQUAREOFF
            if exit_signal(timestamp):
                print("************  MARKET CLOSE  *********** ")
                UpdateDispersionTickers(timestamp, 'NoHedging', False, True, False)
                squareoff(timestamp, f'Exit Square Off', True)
                trade_end_date = timestamp
                trade_count+=1
                zoom_dispersion_trade(trade_start_date, trade_end_date, fr"{folder_path}\Zoom_Trade.xlsx")
                visualise_dispersion_trade(trade_start_date, trade_end_date, fr"{folder_path}\Visualise_Trade.html")
                continue

            # PROFIT TARGET AND STOP LOSS SQUAREOFF
            if TrackPTSL.is_valid(timestamp) and TrackPTSL.status(timestamp) != PTSL.Valid:
                print("************  TrackPTSL Trigger hit *********** ")
                print(f"{TrackPTSL.nature} Square off at NetPnl of {TrackPTSL.pnl_last_trade}")
                UpdateDispersionTickers(timestamp, 'NoHedging', False, True, False)
                squareoff(timestamp, f'{TrackPTSL.nature} Square Off', True)
                trade_end_date = timestamp
                trade_count+=1
                zoom_dispersion_trade(trade_start_date, trade_end_date, fr"{folder_path}\Zoom_Trade.xlsx")
                visualise_dispersion_trade(trade_start_date, trade_end_date, fr"{folder_path}\Visualise_Trade.html")
                continue
                
            # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
            UpdateDispersionTickers(timestamp, 'Delta Hedging using Synthetic Futures', True, True, True)
            print("========================================================================================================================================================================")
            continue
        #######################################################################################################################################################
        
        
        #######################################################################################################################################################
        if entry_signal(timestamp):
            print("************  ENTRY TIME REACHED  ************  ")
            UpdateDispersionTickers(timestamp, "NoHedging", False, False, False)
            if TL.isTodayAnyExpiry(timestamp, *portfolio):
                print("Expiry Day today. Trading on Expiry Day is not Allowed")
                continue

            if not TrackPTSL.is_valid(timestamp):
                print(f"{TrackPTSL.nature} was Triggered at {TrackPTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
                continue
            
            for ticker in portfolio:
                if ticker.is_index:
                    ticker.set_intention(ICView)
                else:
                    ticker.set_intention(ICView.opposite())
                lots_ticker = get_lots_for_entry(ticker, timestamp)
                take_dispersion_position(timestamp, f'{ICView.name} IC', ticker, lots_ticker)

            TrackPTSL.fresh_trade(timestamp)
            current_position = ICView
            trade_start_date = timestamp
            print(f"{ICView} IC Trade executed")

            continue
        #######################################################################################################################################################

        UpdateDispersionTickers(timestamp, "NoHedging", False, True, False)
        print("========================================================================================================================================================================")
        

    file.close()
    sys.stdout = original_stdout.value

In [ ]:
visualise_dispersion_trade(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name='PLOT_Complete_Period_Summary.html'
)

In [ ]:
save_dispersion_trades(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name=f'Info_Trades_for_{strategy_desc}.xlsx'
)

In [ ]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()

In [ ]:
metrics(100000)